In [1]:
import os
import json
import torch
torch.cuda.is_available()
def json_standard(input_object):
    """transfrom python object into json and str with json format 从而增强gpt的成功率
    json.dumps(): 将 Python 对象编码成 JSON 字符串。
    json.dump(): 将 Python 对象编码成 JSON 字符串，并写入到文件中。
    json.loads(): 将已编码的 JSON 字符串解码为 Python 对象。
    json.load(): 读取文件中的 JSON 编码的数据，并解码为 Python 对象。
    """
    # print(input_object,type(input_object))
    return json.dumps(input_object,ensure_ascii=False,indent=4)
def read_file(file_path:str):
    with open(file_path,"r",encoding="utf-8") as i_f:
        return json.load(i_f)
def write_file(file_path:str,obj,overwrite=False):
    if os.path.exists(file_path)==False or overwrite==True:
        with open(file_path,"w",encoding="utf-8") as o_f:
            json.dump(obj,o_f,ensure_ascii=False)
        print(f"写入{file_path}")
    else:
        print(f"{file_path}文件已经存在")
def check_file(file_path:str):
    """返回该路径下是否存在文件"""
    is_exist=os.path.exists(file_path)
    if is_exist:
        file_name=file_path.split("/")[-1]
        print(f"{file_name} exist")
    return is_exist


In [2]:
# 计算message 和string中的token数量
import tiktoken
# 2. Load an encoding 使用tiktoken.get_encoding() 按名称加载编码。第一次运行时，它将需要互联网连接进行下载。后续运行不需要互联网连接。
encoding = tiktoken.get_encoding("cl100k_base")
# 使用tiktoken.encoding_for_model()函数可以自动加载给定模型名称的正确编码。
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# 3. Turn text into tokens with encoding.encode() The .encode() method converts a text string into a list of token integers.
encoding.encode("tiktoken is great!")
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """返回文本字符串中的Token数量"""
    num_tokens=0
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens
def calculate_messages(messages:list) -> int:
    """返回总的message长度"""
    counter=0
    for mess in messages:
        counter+=num_tokens_from_string(mess.content,"cl100k_base")
    return counter
INCONTEXT_WINDOWS_GPT_3_5=16000
INCONTEXT_WINDOWS_MOONSHOT=30000
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI
import os
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage,AIMessage
from langchain_openai import ChatOpenAI
import time
# moonshot_chat.openai_api_base="https://api.moonshot.cn/v1"
# moonshot_chat=ChatOpenAI(temperature=0,model="moonshot-v1-32k",
# moonshot_chat.temperature
class ChatGPTAnnotator:
    def __init__(self):
        self.chat = ChatOpenAI(temperature=0,
                               model="gpt-3.5-turbo",openai_api_key='XXX',
                               openai_api_base='https://gptnb.keqichen.top/v1')
    def retry_chat(self,messages,max_try_times:int=3):
        response=self.chat(messages)
        final_response_content=response.content
        counter=0
        response_metadata={"finish_reason": "stop", "logprobs": None}
        while "length" in response.response_metadata["finish_reason"] and counter<max_try_times:
            # print(len(messages),response)
            messages.append(AIMessage(
                content=final_response_content
            ))
            messages.append(HumanMessage(
                content="Well done! Please go on."
            ))
            response=self.chat(messages)
            final_response_content+=response.content
            counter+=1
            time.sleep(2)
        if counter==max_try_times:
            print(response.response_metadata)
            raise MyException("The response is too long! Max_try!!!!!!!")
        time.sleep(1)
        return final_response_content,response.response_metadata
    def init_theme_summary(self):
        messages=[SystemMessage(
            content="""As a helpful AI and a professional psychotherapist. Please briefly summary the theme of this counseling in 20 words. Please return a json format. There is an example."""),
        HumanMessage(content=json_standard({"theme":"""Schools: individual work In the bustling school environment, individual counselling or coaching sessions give students a precious space for calm reflection. SFBT offers an approach that fits perfectly with the requirement for a time-limited intervention where students don’t have to feel obligated to attend more than they feel is necessary. We have found that half an hour is ample time for a session and the majority of students appreciate the experience and wish to return for follow-up sessions. Other members of staff can participate as appropriate, and group sessions can also be run (see Chapter 79)."""})),
        AIMessage(content=json_standard({"theme": """The importance of individual counseling in schools and SFBT intervention."""}))
        ]
        token_cnt=calculate_messages(messages)
        return messages,token_cnt
    def summary_theme(self,input_json:str):
        messages,token_count=self.init_theme_summary()
        # 记得修改对应的初始化语句
        messages.append(HumanMessage(
            content=input_json 
        ))
        resp,meta_data=self.retry_chat(messages,3)
        return resp,meta_data
    def init_background_message(self):
        messages=[SystemMessage(
            content="As a helpful AI and a professional psychotherapist. Please briefly summary the background of this counseling in 20 words.There is an example."
        ),
        HumanMessage(content=json_standard({"pre_resoning":"Wittgenstein suggests how putting on a “happy face” might be useful when his questioner asks: “How long have you been happy?” A peculiar question. But it might make sense. The answer might be: “Whenever I think about it” (LWPP, p. 2e).","dialog":['Carl: Yeah, but, I put on my happy face. Yeah, that’s what I call it.']})),
        AIMessage(content=json_standard({"background": "The client refers to putting on a 'happy face' as a coping mechanism."}))
        ]
        token_cnt=calculate_messages(messages)
        return messages,token_cnt
    
    def annotate_background(self,input_json:str):
        messages,token_count=self.init_background_message()
        # 记得修改对应的初始化语句
        messages.append(HumanMessage(
            content=input_json 
        ))
        resp,meta_data=self.retry_chat(messages,3)
        return resp,meta_data

    def init_classify_method_message(self):
        system_fewshot={
        'Cognitive Behaviour Therapy': "Client: I've been feeling really anxious lately, especially in social situations.",
        'Solution-Focused Brief Therapy': "Therapist: Can you tell me about a time when the problem wasn't as overwhelming?",
        'Family Therapy': "Therapist: How does your family typically handle conflicts?",
        'Maternal-Infant Interaction and Adult Psychotherapy': "Therapist: How do you feel when you interact with your child?",
        'Multicultural Integrative Therapy': "Therapist: Can you tell me about your cultural background and how it influences your beliefs?",
        'Postmodern Therapy': "Therapist: What's the story you've been telling yourself about this situation?",
        'Marriage and Family Systems Therapy': "Therapist: How do you and your partner communicate during disagreements?",
        'Client-Centered Therapy': "Therapist: What's going through your mind right now?",
        'Existential Therapy': "Therapist: Can you talk about what gives your life meaning and purpose?",
        'Gestalt Therapy': "Therapist: Can you describe your experience in this moment?",
        'Reality Therapy': "Therapist: Let's focus on what you can do right now to change the situation.",
        'Rational Emotive Behavior Therapy': "Therapist: What thoughts are contributing to your current emotions?",
        'Adlerian Therapy': "Therapist: Can you tell me about your earliest memories of feeling this way?",
        'Feminist Therapy': "Therapist: How do societal expectations about gender roles affect your feelings?",
        'Psychoanalytic Therapy': "Therapist: Let's explore your dreams and childhood experiences to understand your current struggles."
        ,
        'Acceptance Commitment Therapy': "Client: I've been trying to avoid my anxiety, but it seems to only make it worse."
        }
        messages=[SystemMessage(
            content="As a helpful AI and a professional psychotherapist. Please classify the method of the dialog. There is the case and example ."+json_standard(system_fewshot)+"return a json with result.And there is an example"
        ),
        HumanMessage(content=json_standard({"dialog":['Therapist: So here I am, your worry/sadness/anger monster in front of you with a rope and I want to play tug of war with you.']})),
        AIMessage(content=json_standard({"reasoning":" the dialogue belongs to Acceptance and Commitment Therapy (ACT) because it utilizes metaphorical language to externalize the client's emotions, presenting them as a 'monster'.","method":"Acceptance Commitment Therapy"})),
        ]
        token_count=calculate_messages(messages)
        return messages,token_count


    def annotate_method(self,input_json:str):
        messages,token_count=self.init_classify_method_message()
        messages.append(HumanMessage(
            content=input_json 
        ))
        resp,meta_data=self.retry_chat(messages,3)
        return resp,meta_data
chat_item=ChatGPTAnnotator()

In [70]:
import os
en_proc_data=[]
def read_files():
    raw_input_dir="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/添加标记case文件夹"
    file_list=sorted(os.listdir(raw_input_dir),key=lambda x:int(x.split("_")[0]))
    for index,file_name in enumerate(file_list[:]):
        input_path=raw_input_dir+"/"+file_name
        input_json=read_file(input_path)
        en_proc_data.append(input_json)
read_files()

In [82]:
cache_path="/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/cache/英文数据/en_regular_cache.json"
if check_file(cache_path):
    en_proc_data=read_file(cache_path)
    

en_regular_cache.json exist


In [131]:
from collections import Counter
method_mapping = {
    'Not Applicable':"Unknown",
    'Not applicable':"Unknown",
    'Not a specific therapy method, as the dialogue does not align with any recognized therapeutic approach':"Unknown",
    'Feminist Therapy': 'Client-Centered Therapy',  # 与 "Client-Centered Therapy" 最相关
    'Psychodynamic Therapy': 'Unknown',  # 与 "Psychoanalytic Therapy" 最相关
    'Multicultural Integrative Therapy': 'Family Therapy',  # 与 "Family Therapy" 最相关
    'Psychotherapy': 'Cognitive Behaviour Therapy',  # 这里我假设 "Psychotherapy" 与 "Client-Centered Therapy" 最相关，但这是一个可能的匹配
    'Dynamic Psychotherapy':'Cognitive Behaviour Therapy'
}
def show_data(cur_data):
    themes=[]
    background=[]
    method=[]
    for idx,item in enumerate(cur_data):
        method_=item["topic_dialog"]["method"]
        if item["theme"] in ["N/A","n/a","Unknown","unknown"]:
            print(item["theme"])
            cur_data[idx]["theme"]=item["topic_dialog"]["method"]
        if method_ in method_mapping.keys():
            cur_data[idx]["topic_dialog"]["method"]= method_mapping[method_]
        if 'Acceptance  Commitment Therapy' in method_:
            cur_data[idx]["topic_dialog"]["method"]='Acceptance Commitment Therapy'
        
        themes.append(item["case_theme"])
        if 'Unknown' in item["background"]:
            background.append(item["background"])
        method.append(item["topic_dialog"]["method"])
    return Counter(themes),Counter(background),Counter(method) 

In [124]:
t,b,m=show_data(en_proc_data)
m

Counter({'Solution-Focused Brief Therapy': 371,
         'Psychoanalytic Therapy': 46,
         'Cognitive Behaviour Therapy': 42,
         'Client-Centered Therapy': 24,
         'Adlerian Therapy': 21,
         'Marriage and Family Systems Therapy': 12,
         'Family Therapy': 11,
         'Acceptance Commitment Therapy': 10,
         'Rational Emotive Behavior Therapy': 8,
         'Reality Therapy': 5,
         'Gestalt Therapy': 5,
         'Existential Therapy': 4,
         'Postmodern Therapy': 3,
         'Maternal-Infant Interaction and Adult Psychotherapy': 1})

In [125]:
t

Counter({'Unknown': 137,
         'Solution-Focused Brief Therapy': 55,
         'The Friendly Stomach Discussion\n': 27,
         'Cognitive Behaviour Therapy': 26,
         'My Real Self\n': 24,
         'Using Your Notes to Construct a Message\n': 21,
         'Appendix B Counseling Session With Kasey: A Practice Exercise\n': 21,
         'I Feel Really Confused\n': 18,
         '2. Exploring the Future Without a Change in the Other\n': 14,
         'INVESTIGATING MARKED MOOD VARIATIONS\n': 9,
         'Therapeutic Principles And Techniques: Main Interventions\n': 9,
         'The Involuntary Client\n': 7,
         'I Feel Really Confused': 6,
         'The Friendly Stomach Discussion': 6,
         'INVESTIGATING MARKED MOOD VARIATIONS': 5,
         'THE INTERVENTION\n': 5,
         'THE ‘TUG OF WAR’ EXERCISE\n': 5,
         'Chapter Seven Working With Reluctant Clients and Challenging Situations\n': 5,
         '3. Recent Changes or Differences\n': 5,
         '3. When the Client S

In [93]:
"""GPT主处理函数"""

def annotate_method_main(cache_data):
    for idx,item in enumerate(cache_data):

        if item["topic_dialog"]["method"] in ["Unknown","unknown",'Unknown']:
            print(idx)
            input_object=[i["participant"]+": "+i["text"] for i in item["topic_dialog"]["dialog"][:10]]
            input_json_str=json_standard(input_object)
            resp,meta=chat_item.annotate_method(input_json_str)
            try:
                resp_json=json.loads(resp)
                method,method_reason=resp_json["method"],resp_json["reasoning"]
            except Exception as e:
                print(resp,meta,e) 
                print("跳过负样本")
                continue
            cache_data[idx]["topic_dialog"]["method"]=method
            cache_data[idx]["extra_info"].append({"method_reasoning":method_reason})
        if item["background"] in ["Unknown","unknown",'Unknown']:
            input_dialog=[i["participant"]+": "+i["text"] for i in item["topic_dialog"]["dialog"][:5]]
            input_object={"pre_reasoning":item["pre_reasoning"],"dialog":input_dialog}
            resp,meta=chat_item.annotate_background(json_standard(input_object))
            try:
                resp_json=json.loads(resp)
                background=resp_json["background"]
            except Exception as e:
                print(resp,meta,e,"跳过负样本")
                continue
            cache_data[idx]["background"]=background
            print(idx,background)

cache_data=en_proc_data.copy()
annotate_method_main(cache_data)
            
            

In [84]:
a,b,m=show_data(en_proc_data)
m

Counter({'Solution-Focused Brief Therapy': 371,
         'Psychoanalytic Therapy': 46,
         'Cognitive Behaviour Therapy': 42,
         'Client-Centered Therapy': 24,
         'Adlerian Therapy': 21,
         'Marriage and Family Systems Therapy': 12,
         'Family Therapy': 11,
         'Acceptance Commitment Therapy': 10,
         'Rational Emotive Behavior Therapy': 8,
         'Reality Therapy': 5,
         'Gestalt Therapy': 5,
         'Existential Therapy': 4,
         'Postmodern Therapy': 3,
         'Maternal-Infant Interaction and Adult Psychotherapy': 1})

写入/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/cache/英文数据/en_regular_cache3.json


In [98]:
c,b,m=show_data(en_proc_data)
# c,b,m

In [99]:
c

Counter({'Unknown': 137,
         'Solution-Focused Brief Therapy': 55,
         'The Friendly Stomach Discussion\n': 27,
         'Cognitive Behaviour Therapy': 26,
         'My Real Self\n': 24,
         'Using Your Notes to Construct a Message\n': 21,
         'Appendix B Counseling Session With Kasey: A Practice Exercise\n': 21,
         'I Feel Really Confused\n': 18,
         '2. Exploring the Future Without a Change in the Other\n': 14,
         'INVESTIGATING MARKED MOOD VARIATIONS\n': 9,
         'Therapeutic Principles And Techniques: Main Interventions\n': 9,
         'The Involuntary Client\n': 7,
         'I Feel Really Confused': 6,
         'The Friendly Stomach Discussion': 6,
         'INVESTIGATING MARKED MOOD VARIATIONS': 5,
         'THE INTERVENTION\n': 5,
         'THE ‘TUG OF WAR’ EXERCISE\n': 5,
         'Chapter Seven Working With Reluctant Clients and Challenging Situations\n': 5,
         '3. Recent Changes or Differences\n': 5,
         '3. When the Client S

In [ ]:
themes=c.keys()


In [140]:
beg_list=["case","example","case study","chapter","appendix b"]
tmp_show=[]
def regular_en_data(cur_data):
    for idx,item in enumerate(cur_data):
        theme=item["case_theme"]
        if theme in ['Unknown','unknown']:
            theme=item["topic_dialog"]["method"]
            # cur_data[idx]["topic_dialog"]["method"]
        theme=theme.lower()
        while True:
            origin_theme=theme
            if theme!="" and not theme[0].isalpha():
                theme=theme[1:]
            if theme!="" and not theme[-1].isalpha() and theme[-1]!="?":
                theme=theme[:-1]
            for target in beg_list:
                if theme.startswith(target):
                    theme=theme[len(target):]
            if origin_theme==theme:
                break
        # tmp_show.append(theme)
        if len(theme)>50:
            resp,meta=chat_item.summary_theme(json_standard({"theme":item["pre_reasoning"]}))
            resp_json=json.loads(resp)
            theme=resp_json["theme"]
            print(idx,theme)
        elif theme.endswith(" therapy"):
            resp,meta=chat_item.summary_theme(json_standard({"theme":item["pre_reasoning"]}))
            resp_json=json.loads(resp)
            theme=theme+": "+resp_json["theme"]
            print(idx,theme)
        cur_data[idx]["theme"]=theme.lower()
    return cur_data
cache_data=en_proc_data.copy()
theme_data=regular_en_data(cache_data)
# Counter(tmp_show)

0 cognitive behaviour therapy: Exploring cognition through images and thoughts in therapy to uncover emotionally charged thoughts and beliefs.
1 cognitive behaviour therapy: Exploring inner struggles through imagery, addressing challenges when clients respond with 'I don't know' in therapy.
2 cognitive behaviour therapy: Balancing therapist suggestions to promote client introspection and avoid passivity, assessing responses, and suggesting opposite thoughts for exploration.
3 cognitive behaviour therapy: Exploring implicit meanings in client communication and the importance of uncovering unexpressed meanings in therapy.
4 cognitive behaviour therapy: Identifying and modifying 'hot thoughts' in therapy through collaborative exploration and vivid imagination of problematic situations.
5 cognitive behaviour therapy: Exploring and addressing clients' fears and avoidance behaviors through situational exposure in therapy sessions.
6 cognitive behaviour therapy: Experiential exposure, role-pl

In [142]:
[item["theme"] for item in theme_data]

['cognitive behaviour therapy: exploring cognition through images and thoughts in therapy to uncover emotionally charged thoughts and beliefs.',
 "cognitive behaviour therapy: exploring inner struggles through imagery, addressing challenges when clients respond with 'i don't know' in therapy.",
 'cognitive behaviour therapy: balancing therapist suggestions to promote client introspection and avoid passivity, assessing responses, and suggesting opposite thoughts for exploration.',
 'cognitive behaviour therapy: exploring implicit meanings in client communication and the importance of uncovering unexpressed meanings in therapy.',
 "cognitive behaviour therapy: identifying and modifying 'hot thoughts' in therapy through collaborative exploration and vivid imagination of problematic situations.",
 "cognitive behaviour therapy: exploring and addressing clients' fears and avoidance behaviors through situational exposure in therapy sessions.",
 'cognitive behaviour therapy: experiential expos

写入/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/cache/英文数据/en_regular_cache3.json


NameError: name 'theme_data' is not defined

In [147]:

def order_data(old_data):
    result=[]
    topic_order=['theme','is_same_qa', 'is_same_case', 'background', 'pre_reasoning', 'topic_dialog', 'post_reasoning' ]
    dialog_order=['key','unit_id','method','dialog']
    for idx,item in enumerate(old_data):
        item['topic_dialog'] = {key:item['topic_dialog'][key] for key in dialog_order}
        ordered_data = {key: item[key] for key in topic_order}
        result.append(ordered_data)
    return result
sorted_data=order_data(theme_data)

In [155]:
def regular_reasoning(cur_data):
    for idx,item in enumerate(cur_data):
        if type(item['pre_reasoning'])!=str:
           cur_data[idx]['pre_reasoning']="".join(item['pre_reasoning'])
        if type(item['post_reasoning'])!=str:
            cur_data[idx]['post_reasoning']="".join(item['post_reasoning'])
    return cur_data
regular_data=regular_reasoning(sorted_data.copy())

In [156]:
regular_data[]

[{'theme': 'cognitive behaviour therapy: exploring cognition through images and thoughts in therapy to uncover emotionally charged thoughts and beliefs.',
  'is_same_qa': 0,
  'is_same_case': 0,
  'background': 'A client experiences anxiety while walking down the high street due to negative self-perceptions.',
  'pre_reasoning': 'The term ‘cognition’ includes images as well as thoughts. Some clients might find it difficult to pinpoint their NATs in a specific situation, but the therapist can ask such clients if they had any pictures or images in their minds in that situation which might be more retrievable. Using imagery can help clients to relive past situations in order to uncover the hot (emotionally charged) thoughts associated with these situations.',
  'topic_dialog': {'key': 'fewshot',
   'unit_id': 0,
   'method': 'Cognitive Behaviour Therapy',
   'dialog': [{'id': '0-0',
     'participant': 'Client',
     'text': ' I’m walking down the high street. I feel very uncomfortable. I

In [157]:
write_file("/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/cache/英文数据/en_regular_cache4.json",regular_data,True)

写入/home/ckqsudo/code2024/0dataset/E-bed/English/z3_gpt_mapping.ipynb/cache/英文数据/en_regular_cache4.json


In [148]:
sorted_data[0]

{'theme': 'cognitive behaviour therapy: exploring cognition through images and thoughts in therapy to uncover emotionally charged thoughts and beliefs.',
 'is_same_qa': 0,
 'is_same_case': 0,
 'background': 'A client experiences anxiety while walking down the high street due to negative self-perceptions.',
 'pre_reasoning': 'The term ‘cognition’ includes images as well as thoughts. Some clients might find it difficult to pinpoint their NATs in a specific situation, but the therapist can ask such clients if they had any pictures or images in their minds in that situation which might be more retrievable. Using imagery can help clients to relive past situations in order to uncover the hot (emotionally charged) thoughts associated with these situations.',
 'topic_dialog': {'key': 'fewshot',
  'unit_id': 0,
  'method': 'Cognitive Behaviour Therapy',
  'dialog': [{'id': '0-0',
    'participant': 'Client',
    'text': ' I’m walking down the high street. I feel very uncomfortable. I think ever

In [132]:
show_data(en_proc_data)

(Counter({'Unknown': 137,
          'Solution-Focused Brief Therapy': 55,
          'The Friendly Stomach Discussion\n': 27,
          'Cognitive Behaviour Therapy': 26,
          'My Real Self\n': 24,
          'Using Your Notes to Construct a Message\n': 21,
          'Appendix B Counseling Session With Kasey: A Practice Exercise\n': 21,
          'I Feel Really Confused\n': 18,
          '2. Exploring the Future Without a Change in the Other\n': 14,
          'INVESTIGATING MARKED MOOD VARIATIONS\n': 9,
          'Therapeutic Principles And Techniques: Main Interventions\n': 9,
          'The Involuntary Client\n': 7,
          'I Feel Really Confused': 6,
          'The Friendly Stomach Discussion': 6,
          'INVESTIGATING MARKED MOOD VARIATIONS': 5,
          'THE INTERVENTION\n': 5,
          'THE ‘TUG OF WAR’ EXERCISE\n': 5,
          'Chapter Seven Working With Reluctant Clients and Challenging Situations\n': 5,
          '3. Recent Changes or Differences\n': 5,
          '

In [65]:


common_method = [
    'Solution-Focused Brief Therapy',
    'Psychoanalytic Therapy',
    'Cognitive Behaviour Therapy',
    'Adlerian Therapy',
    'Marriage and Family Systems Therapy',
    'Client-Centered Therapy',
    'Acceptance Commitment Therapy',
    'Family Therapy',
    'Rational Emotive Behavior Therapy',
    'Reality Therapy',
    'Existential Therapy',
    'Gestalt Therapy',
    'Postmodern Therapy'
]

In [135]:
len(en_proc_data)

563

In [46]:
cn_raw_data=read_file("/home/ckqsudo/code2024/0dataset/E-bed/Chinese/中文数据_处理错reasoning_分离长对话数据_分离label_对话人.json")

In [47]:
cn_en_method={
'认知行为疗法': 'Cognitive Behaviour Therapy',
'SFBT': 'Solution-Focused Brief Therapy',
'母婴互动及成人心理治疗': 'Maternal-Infant Interaction and Adult Psychotherapy',
'多元文化整合疗法': 'Multicultural Integrative Therapy',
'后现代主义疗法': 'Postmodern Therapy',
'家庭疗法': 'Family Therapy',
'Group Dialog': 'Family Therapy',
'婚姻与家庭系统疗法': 'Marriage and Family Systems Therapy',
'来访者中心疗法': 'Client-Centered Therapy',
'存在主义疗法': 'Existential Therapy',
'现实疗法': 'Reality Therapy',
'理性情绪行为法': 'Rational Emotive Behavior Therapy',
'格式塔疗法': 'Gestalt Therapy',
'阿德勒疗法': 'Adlerian Therapy',
'女权主义疗法': 'Feminist Therapy',
'精神分析疗法': 'Psychoanalytic Therapy',
'完型治疗': 'Gestalt Therapy'
}

for idx,item in enumerate(cn_raw_data):
    # print(item.keys())
    if 'method' in item.keys():
        
        if item['method'] in ['认知疗法','行为主义疗法']:
            cn_raw_data[idx]['method']='认知行为疗法'
        if item['method'] in ['双人对话','']:
            cn_raw_data[idx]['method']="2人对话"
        if item['method'] in ['3人对话']:
            cn_raw_data[idx]['method']="多人对话"
        if item['method'] in ['认知行为疗法(多人对话)']:
            cn_raw_data[idx]['method']="Group Dialog"
        method_=cn_raw_data[idx]['method']
        if method_ in cn_en_method.keys():
            cn_raw_data[idx]['method']=cn_en_method[method_]

    # print(item)
cn_method=[]
for idx,item in enumerate(cn_raw_data):
    cn_method.append(item['method'])
Counter(cn_method)

Counter({'Cognitive Behaviour Therapy': 415,
         'Unknown': 197,
         'Solution-Focused Brief Therapy': 159,
         '2人对话': 139,
         'Family Therapy': 105,
         'Maternal-Infant Interaction and Adult Psychotherapy': 87,
         'Multicultural Integrative Therapy': 78,
         'Postmodern Therapy': 57,
         'Marriage and Family Systems Therapy': 37,
         'Client-Centered Therapy': 31,
         'Existential Therapy': 29,
         'Gestalt Therapy': 22,
         'Reality Therapy': 21,
         'Rational Emotive Behavior Therapy': 17,
         'Adlerian Therapy': 14,
         'Feminist Therapy': 14,
         'Psychoanalytic Therapy': 12,
         '多人对话': 10})

In [48]:
write_file("/home/ckqsudo/code2024/0dataset/E-bed/中文快速处理工作流/flow1/中文原始数据.json",cn_raw_data)

/home/ckqsudo/code2024/0dataset/E-bed/中文快速处理工作流/flow1/中文原始数据.json文件已经存在
